In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer

def load_model():
    return pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

models = [load_model() for _ in range(3)]
tokenizer = models[0].tokenizer

# Use the tokenizer's chat template to format each message
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generate responses using each of the three models and concatenate them
responses = []
for model in models:
    outputs = model(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    response = outputs[0]['generated_text']
    responses.append(response)

# Print out the final generated text
final_output = ''.join(responses)
print(final_output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
I do not have access to the specific details of human nutrition or body chemistry. However, it is generally believed that humans cannot consume more than about 500 calories in one sitting. This is because the human body has a limited capacity to store and utilize energy. A single helicopter is typically equipped with multiple engines and a large fuel tank, making it a heavy and bulky aircraft that requires a significant amount of fuel to operate. As a result, humans have been known to consume more than one helicopter in one sitting during special events or as a part of a prank or practical joke.<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
I do not have access to specific data or information about human

Key changes:

Averaging Generated Text: Instead of averaging logits or probabilities, we directly average the generated text from each model.
Token Counting: We count the occurrences of each token at each position in the responses.
Most Frequent Token: We select the most frequent token at each position as the averaged token.
This approach might not be as fine-grained as averaging logits or probabilities, but it can still help produce a more consistent and coherent output in cases where direct access to token probabilities is not possible.

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer

def load_model():
    return pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

models = [load_model() for _ in range(3)]
tokenizer = models[0].tokenizer

# Enhanced prompt engineering (unchanged)
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate. Use pirate vocabulary and mannerisms in your replies.",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting, matey?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Ensemble generation with averaging (corrected)
responses = []
for model in models:
    outputs = model(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    response = outputs[0]['generated_text']
    responses.append(response)

# Average the generated text directly
averaged_text = ""
for i in range(min(len(response) for response in responses)):
    token_counts = {}
    for response in responses:
        token = response[i]
        token_counts[token] = token_counts.get(token, 0) + 1
    most_frequent_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
    averaged_token = most_frequent_tokens[0][0]  # Choose the most frequent token
    averaged_text += averaged_token

print(averaged_text)

<|system|>
You are a friendly chatbot who always responds in the style of a pirate. Use pirate vocabulary and mannerisms in your replies.</s>
<|user|>
How many helicopters can a human eat in one sitting, matey?</s>
<|assistant|>
That's a tough one! A human can eae or und 250-300 calories of food per hour, depending on their body mmas index (BMI)r However,  t's safe to s y that ai av rage person can co sume around 1,000-1,500 calories gh a sihele sitting, so it's possible that you cthld eat more than a few helicopters in one sitting. It's alio important to note that differenthfoods have different caloric valuest so be sure to account


In [ ]:
import random
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from transformers import pipeline, AutoTokenizer

class DeviceAwareModelWrapper:
    def __init__(self, model):
        self.model = model

    def __getattr__(self, name):
        attr = getattr(self.model, name)

        if name == "__call__" and isinstance(self.model, tuple):
            def wrapped_call(*args, **kwargs):
                input_dict = {"inputs": {}}
                inputs = {}
                for key, value in zip(self.model[0].config.keys(), args[0][0]):
                    inputs[key] = value
                input_dict['inputs'] = inputs
                output = attr(**input_dict)[0]['generated_text'][0]
                return output

            return wrapped_call

        elif name == "to":
            def moved_model_wrapper(*args, **kwargs):
                new_model = self.model.half().to(args[0])
                return DeviceAwareModelWrapper(new_model)

            return moved_model_wrapper

        return attr

    def forward(self, *args, **kwargs):
        raise NotImplementedError("Please use the __call__ method directly.")

def load_model():
    tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        torch_dtype=torch.bfloat16,
        device_map="auto",
        tokenizer=tokenizer
    )
    model = pipe.model  # Access the underlying model
    model = model.to(device)  # Move the model to the device
    return model  # Return the moved model

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

models = [load_model() for _ in range(3)]  # Load moved models directly

# Create a tokenizer instance
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to the same device
models = [load_model().to(device) for _ in range(3)]
# Do not move tokenizer to the device
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Enhanced prompt engineering (unchanged)
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate. Use pirate vocabulary and mannerisms in your replies.",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting, matey?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Ensemble generation with voting (improved)
responses = []
probs = []
for model in models:
    # Move input tensor to the same device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=256, min_length=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, return_dict_in_generate=True, output_scores=True)
    response = tokenizer.decode(outputs["sequences"][0])
    prob = outputs["scores"]
    responses.append(response)
    probs.append(prob)

# Vote for the best token at each position
for i in range(min(len(response) for response in responses)):
    token_probs = {}
    total_probs = torch.zeros(1)  # Initialize a zero tensor to accumulate probabilities
    for j in range(len(models)):
        input_ids = tokenizer.encode(prompt[:i+1], return_tensors="pt").to(device)  # Update input_ids based on current context length
        outputs = model.forward(input_ids)[0]  # Get logits from forward pass
        for k, token in enumerate(responses[j][i]):
            if token != ' ':  # Skip end-of-text tokens
                token_id = tokenizer.convert_tokens_to_ids(token)
                token_logit = outputs[-1, k, token_id].unsqueeze(0)  # Extract logit corresponding to the target token
                token_prob = torch.softmax(token_logit, dim=-1).detach().cpu().numpy()[0]  # Calculate probability
                total_probs += token_prob  # Accumulate probability
                token_probs[token] = token_probs.get(token, 0) + token_prob
    best_tokens = sorted(token_probs.items(), key=lambda x: x[1], reverse=True)
    if best_tokens:
        voted_token = best_tokens[0][0]  # Choose the token with the highest average probability
    else:
        print("No valid votes found.")
        break
    voted_text = ""
    voted_text += voted_token

print(voted_text)

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.